# Fune-tunning Llama2

In [2]:
!pip3 install transformers>=4.32.0 optimum>=1.12.0
!pip3 install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/  # Use cu117 if on CUDA 11.7

Looking in indexes: https://pypi.org/simple, https://huggingface.github.io/autogptq-index/whl/cu118/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 4.9 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 8.7 MB/s eta 0:00:00:00:0100:01


In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name_or_path = "TheBloke/Llama-2-70B-chat-GPTQ"
# To use a different branch, change revision
# For example: revision="main"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

/u/s/d/sdam/miniconda3/envs/llama2/lib/python3.12/site-packages/transformers/modeling_utils.py:4225: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
The cos_cached attribute will be removed in 4.39. Bear in mind that its contents changed in v4.38. Use the forward method of RoPE from now on instead. It is not used in the `LlamaAttention` class
The sin_cached attribute will be removed in 4.39. Bear in mind that its contents changed in v4.38. Use the forward method of RoPE from now on instead. It is not used in the `LlamaAttention` class


In [2]:
prompt = "Tell me about AI"
prompt_template=f'''[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>
{prompt}[/INST]

'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))

# Inference can also be done using transformers' pipeline

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

print(pipe(prompt_template)[0]['generated_text'])



*** Generate:


In [4]:
prompt = """
Column Names MUST be limited to the following:
name, description, team, type, age, location, year, city, rank, status, state, category,
weight, code, club, artist, result, position, country, notes, class, company, album, symbol,
address, duration, format, county, day, gender, industry, language, sex, product, jockey,
region, area, service, teamName, order, isbn, fileSize, grades, publisher, plays, origin,
elevation, affiliation, component, owner, genre,  manufacturer, brand, family, credit, depth,
classification, collection, species, command, nationality, currency, range, affiliate,
birthDate, ranking, capacity, birthPlace, person, creator, operator, religion, education,
requirement, director, sales, continent, organisation
Do not use any column names aside from these.

Given the following relational table:

IDI1,isopentenyl-diphosphate delta isomerase..isopentenyl-diphosphate delta isomerase.
FDFT1,farnesyl-diphosphate farnesyltransferase 1.DGPT.farnesyl-diphosphate farnesyltransferase 1.
FDPS,"farnesyl diphosphate synthase (farnesyl pyrophosphate synthetase, dimethylallyltranstransferase, geranyltranstransferase).FPS.."
HPCAL1,hippocalcin-like 1.BDR1; HLP2; VILIP-3.hippocalcin-like 1.
LRP8,"low density lipoprotein receptor-related protein 8, apolipoprotein e receptor.APOER2; HSZ75190.low density lipoprotein receptor-related protein 8 isoform 3 precursor."
AP2S1,"adaptor-related protein complex 2, sigma 1 subunit.AP17; CLAPS2; AP17-DELTA.adaptor-related protein complex 2, sigma 1 subunit isoform AP17delta."
SQLE,squalene epoxidase..squalene monooxygenase.
HPCA,hippocalcin.BDR2.hippocalcin.
,"tubulin, beta, 2."
AP3S1,"adaptor-related protein complex 3, sigma 1 subunit.CLAPS3; Sigma3A.adaptor-related protein complex 3, sigma 1 subunit."
AP1S2,"adaptor-related protein complex 1, sigma 2 subunit.DC22; SIGMA1B; MGC:1902.adaptor-related protein complex 1 sigma 2 subunit."
TUBA1,"tubulin, alpha 1 (testis specific).FLJ30169; H2-ALPHA.tubulin, alpha 1."
AP2A1,"adaptor-related protein complex 2, alpha 1 subunit.ADTAA; CLAPA1; AP2-ALPHA.adaptor-related protein complex 2, alpha 1 subunit isoform 2."
SEC24D,"SEC24 related gene family, member D (S. cerevisiae).KIAA0755.Sec24-related protein D."

Guess the column names for the whole table. There are only 2 columns in the table. Give only 1 answer for each column.
"""
prompt_template=f'''[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
All your answers should be in the format of a list, such as [col1,col2,...].
<</SYS>>
{prompt}[/INST]

'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=4096)
print(tokenizer.decode(output[0]))



*** Generate:


NameError: name 'tokenizer' is not defined

In [2]:
import numpy as np
import os
import pandas as pd
from json import loads
from tqdm import tqdm

In [15]:
prompt = """
    Column Names are limited to the following:
    name, description, team, type, age, location, year, city, rank, status, state, category,
    weight, code, club, artist, result, position, country, notes, class, company, album, symbol,
    address, duration, format, county, day, gender, industry, language, sex, product, jockey,
    region, area, service, teamName, order, isbn, fileSize, grades, publisher, plays, origin,
    elevation, affiliation, component, owner, genre,  manufacturer, brand, family, credit, depth,
    classification, collection, species, command, nationality, currency, range, affiliate,
    birthDate, ranking, capacity, birthPlace, person, creator, operator, religion, education,
    requirement, director, sales, continent, organisation
    Do not use any column names aside from these.

    Output must be in valid JSON like the following example {"colnames" : ["col1", "col2"]}
    """

tabledir = "K4" # Note that this doesn't contain '/' at the end
filenames = os.listdir(tabledir + '/')
real_cols = []
pred_cols = []

for filename in tqdm(filenames[0:100]):
    with open(tabledir + '/' + filename) as f:
        linelist = f.readlines()
        colnames = linelist[0][:-1].split(',')
        real_cols += colnames

        prompt_template=f'''[INST] <<SYS>>
You are a database expert who can make general predictions for missing column values in database tables, and the predicted column names are within the required candidate set. All output must be in valid JSON. Don't add explanation beyond the JSON.
Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>
{prompt}
Given the following relational table:
{''.join(linelist[1:21])}
Guess the column names for the whole table. There are only {len(colnames)} columns in the table. It is possible for multiple columns to have the same name
[/INST]
'''

        #print("*** Generate:")
        input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
        output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=4096)
        #print(tokenizer.decode(output[0]).replace('\n',' '))
        pred_tmp = loads(tokenizer.decode(output[0]).split('[/INST]')[1].split('</s>')[0].replace('\n',''))["colnames"]
        if len(colnames) == len(pred_tmp): # TODO: add an else statement
            pred_cols += pred_tmp
        else:
            pred_cols += ["???"] * len(colnames)

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [06:08<00:00,  3.68s/it]


In [16]:
with open(tabledir + '_true.npy', 'wb') as f:
    np.save(f, np.array(real_cols, dtype='<U14'))
with open(tabledir + '_pred.npy', 'wb') as f:
    np.save(f, np.array(pred_cols, dtype='<U14'))

In [49]:
prompt = """
    Column Names are limited to the following:
    name, description, team, type, age, location, year, city, rank, status, state, category,
    weight, code, club, artist, result, position, country, notes, class, company, album, symbol,
    address, duration, format, county, day, gender, industry, language, sex, product, jockey,
    region, area, service, teamName, order, isbn, fileSize, grades, publisher, plays, origin,
    elevation, affiliation, component, owner, genre,  manufacturer, brand, family, credit, depth,
    classification, collection, species, command, nationality, currency, range, affiliate,
    birthDate, ranking, capacity, birthPlace, person, creator, operator, religion, education,
    requirement, director, sales, continent, organisation
    Do not use any column names aside from these.

    Output must be in valid JSON like the following example {"column" : "col1"}. Give only 1 prediction. Do NOT add any explanation beyond the JSON.
    """

tabledir = "K0" # Note that this doesn't contain '/' at the end
filenames = os.listdir(tabledir + '/')
real_cols_single = []
pred_cols_single = []

for filename in tqdm(filenames[0:100]):
    fullpath = tabledir + '/' + filename
    with open(fullpath) as f:
        linelist = f.readlines()
        colnames = linelist[0][:-1].split(',')
        real_cols_single += colnames

        df = pd.read_csv(fullpath).astype(str)
        for col in df.columns:
            prompt_template=f'''[INST] <<SYS>>
You are a database expert who can make general predictions for missing column values in database tables, and the predicted column names are within the required candidate set. All output must be in valid JSON. Do NOT add explanation beyond the JSON.
Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>
{prompt}
Given the following column values in a relational table: {', '.join(df[col][0:20])}
Guess the column name
[/INST]
'''

            #print("*** Generate:")
            input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
            output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=4096)
            #print(tokenizer.decode(output[0]).replace('\n',' '))
            pred_cols_single.append(loads('{' + tokenizer.decode(output[0]).split('[/INST]')[1].split('</s>')[0].replace('\n','').replace('}','{').split('{')[1] + '}')["column"])

100%|██████████| 100/100 [12:34<00:00,  7.55s/it]


In [50]:
with open(tabledir + '_true_single.npy', 'wb') as f:
    np.save(f, np.array(real_cols_single, dtype='<U14'))
with open(tabledir + '_pred_single.npy', 'wb') as f:
    np.save(f, np.array(pred_cols_single, dtype='<U14'))